In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tifffile import imread
%matplotlib qt 
import hyperspy.api as hs

In [3]:
img_path = "H:/workspace/p01_code/deep_ptycho/ptyrad/output/CNS/20240610_full_N7134_dp164_random32_p2_1obj_1slice_plr5e-4_oalr5e-4_oplr5e-4_slr5e-4_tlr0_opos_sng1.0_spr0.1/objp_iter0100.tif"
#img_path = "H:/workspace/p01_code/deep_ptycho/ptyrad/output/multi_obj/full_N9494_dp180_sparse64_p2_plr5e-4_olr5e-4_slr5e-4_4obj_1slice/objp_ostack_iter0050.tif"
# img_path = "output/multi_obj_WSe2_and_Cu_4obj_1slice_1e-3_dppow0.5_b64/objp_iter0100.tif"
#img_path = "output/CNS_1obj_1slice_1e-3_revisit/objp_iter0500_5e-4_dppow0.5.tif"
#img_path = "output/CNS_32obj_1slice_5e-4_revisit/objp_iter0500.tif"
#img_path = "output/abtem/potential_256_phonon_0.1_std_zsum_dx0.08Ang.tif"
img = imread(img_path)
img.shape

(392, 403)

In [5]:
# window = 144
# img_crop = img[:,202-window//2:202+window//2, 201-window//2:201+window//2]

window = 144
img_crop = img[202-window//2:202+window//2, 201-window//2:201+window//2]

fig, axs = plt.subplots(1,2)
im00 = axs[0].imshow(np.mean(img_crop, 0))
fig.colorbar(im00, shrink=0.6)
axs[0].set_title(f'Mean image (N={len(img_crop)})')
im01 = axs[1].imshow(np.std(img_crop,0))
fig.colorbar(im01, shrink=0.6)
axs[1].set_title(f'Std image (N={len(img_crop)})')

plt.show()

TypeError: Invalid shape (144,) for image data

# Atomap fitting / model

In [6]:
import atomap.api as am

s = hs.signals.Signal2D(img)
s_peaks = am.get_feature_separation(s, separation_range=(5, 7), show_progressbar=False)
s_peaks.plot()

ModuleNotFoundError: No module named 'atomap'

In [ ]:
atom_positions = am.get_atom_positions(s, separation=6)
sublattice = am.Sublattice(atom_positions, image=s.data)

sublattice.find_nearest_neighbors()
sublattice.refine_atom_positions_using_center_of_mass()
sublattice.refine_atom_positions_using_2d_gaussian(mask_radius=4)

model_image = sublattice.get_model_image()
s.plot(title='Data')
model_image.plot(title='Model')
(s-model_image).plot(title='Difference', cmap='seismic', vmin=-0.2, vmax=0.2)

In [ ]:
plt.figure()
plt.hist(sublattice.atom_amplitude_gaussian2d.ravel(), bins=100)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def generate_gaussian(A, x, y, sx, sy, rotation, size):
    # Generate a grid of coordinates
    xx, yy = np.meshgrid(np.arange(size[1]), np.arange(size[0]))

    # Rotation parameters
    x0 = x
    y0 = y
    theta = rotation 

    # Calculate terms used in the Gaussian function
    sx2 = sx ** 2
    sy2 = sy ** 2
    cos_theta2 = np.cos(theta) ** 2
    sin_theta2 = np.sin(theta) ** 2
    sin_2theta = np.sin(2 * theta)

    a = cos_theta2 / (2 * sx2) + sin_theta2 / (2 * sy2)
    b = -sin_2theta / (4 * sx2) + sin_2theta / (4 * sy2)
    c = sin_theta2 / (2 * sx2) + cos_theta2 / (2 * sy2)

    # Apply rotation and generate Gaussian blob
    x_rot = xx - x0
    y_rot = yy - y0
    gaussian_blob = (
        A
        * (1 / (sx * sy * (2 * np.pi)))
        * np.exp(-(
            a * (x_rot ** 2) + 
            2 * b * (x_rot * y_rot) + 
            c * (y_rot ** 2)
        ))
    )

    return gaussian_blob

def generate_canvas(gaussian_params, canvas_size):
    canvas = np.zeros(canvas_size)
    for params in gaussian_params:
        A, x, y, sx, sy, rotation = params
        gaussian_blob = generate_gaussian(A, x, y, sx, sy, rotation, canvas_size)
        canvas += gaussian_blob
    return canvas

canvas_size = s.data.shape

shift_vec = np.zeros((len(sublattice.atom_list),2)) # np.random.normal(0, 1, size=(len(sublattice.atom_list), 2))
target_indices = np.where(sublattice.atom_amplitude_gaussian2d<7.5)[0]
shift_vec[target_indices] = np.random.normal(0, 1.66, size=(len(target_indices), 2))

sigma_shrink = np.ones(len(sublattice.atom_list))
sigma_shrink[target_indices] = 0.5

# Example Gaussian parameters (x, y, sx, sy, rotation)
gaussian_params = [(sublattice.atom_amplitude_gaussian2d[i],
                    sublattice.x_position[i] ,
                    sublattice.y_position[i] + shift_vec[i,0],
                    sublattice.sigma_x[i] * sigma_shrink[i],
                    sublattice.sigma_y[i] * sigma_shrink[i],
                    sublattice.rotation[i]) for i in range(len(sublattice.atom_list))]



# Generate canvas with Gaussian blobs
perturbed_model = generate_canvas(gaussian_params, canvas_size)

# Plot canvas with Gaussian blobs
plt.figure(figsize=(8, 6))
plt.imshow(model_image, cmap='gray')
plt.title('Model')
plt.colorbar()
plt.axis('off')
plt.show()

plt.figure(figsize=(8, 6))
plt.imshow(perturbed_model, cmap='gray')
plt.title('perturbed_model')
plt.colorbar()
plt.axis('off')
plt.show()

plt.figure(figsize=(8, 6))
plt.title("Model - perturbed model")
plt.imshow(model_image - perturbed_model, cmap='seismic', vmin=-0.2, vmax=0.2)
plt.colorbar()
plt.axis('off')
plt.show()


In [ ]:
# Generate an image stack

omode = 64
imstack = np.zeros((omode,1,391,403), dtype='float32')
target_indices = np.where(sublattice.atom_amplitude_gaussian2d<7.5)[0]
sigma_shrink = np.ones(len(sublattice.atom_list))
sigma_shrink[target_indices] = 0.5

# Prepopulate the gaussian_params
gaussian_params = np.zeros((len(sublattice.atom_list), 6))

for i in range(len(sublattice.atom_list)):
    gaussian_params[i] = (sublattice.atom_amplitude_gaussian2d[i],
                        sublattice.x_position[i],
                        sublattice.y_position[i],
                        sublattice.sigma_x[i],
                        sublattice.sigma_y[i],
                        sublattice.rotation[i])

for ii in range(omode):
    print(f"Generating {ii}")
    shift_vec = np.zeros((len(sublattice.atom_list),2)) # np.random.normal(0, 1, size=(len(sublattice.atom_list), 2))
    shift_vec[target_indices] = np.random.normal(0,1.66, size=(len(target_indices), 2))
    
    perturbed_gaussian_params = gaussian_params.copy()
    #perturbed_gaussian_params[target_indices,1] += shift_vec[target_indices,1] # x_pos
    perturbed_gaussian_params[target_indices,2] += shift_vec[target_indices,0] # y_pos 
    perturbed_gaussian_params[target_indices,3] *= sigma_shrink[target_indices]
    perturbed_gaussian_params[target_indices,4] *= sigma_shrink[target_indices] 

    imstack[ii,0] = generate_canvas(perturbed_gaussian_params, canvas_size)

In [ ]:
hs.signals.Signal2D(imstack[:,0]).plot()
data_arr = np.array(hs.signals.Signal2D(imstack[:,0]))

In [ ]:
plt.figure()
plt.imshow(data_arr.mean(0))

In [ ]:
from tifffile import imwrite
imwrite("output/CNS_64obj_initial_guess_16bit.tif", np.uint16(65535*(data_arr - data_arr.min())/(data_arr.max() - data_arr.min())))
imwrite("output/CNS_64obj_initial_guess_32bit.tif", data_arr)


# hyerspy for SVD approaches

In [ ]:
s = hs.signals.Signal2D(img_crop)
s.plot()
s.decomposition(algorithm="SVD", centre='signal', output_dimension=4) # Default is SVD, other options: PCA, MLPCA, RPCA, ORPCA, NMF, ORNMF,  
s.plot_explained_variance_ratio() 
s.plot_decomposition_results()

In [ ]:
s.plot_decomposition_loadings(4)

In [ ]:
s.plot_decomposition_factors(4)

In [ ]:
sm = s.get_decomposition_model(components=2)
sm.plot(title='model')
(s-sm).plot(cmap='seismic', title='difference')

In [ ]:
s.get_decomposition_model()

In [ ]:
data_arr = np.array(s.data)
model_arr = np.array(sm)

In [ ]:
imwrite("output/CNS_64obj_5e-4_random/CNS_64obj_5e-4_random_16bit.tif", np.uint16(65535*(data_arr - data_arr.min())/(data_arr.max() - data_arr.min())))
imwrite("output/CNS_64obj_5e-4_random/CNS_64obj_5e-4_random_SVD_3_of_13comp_16bit.tif", np.uint16(65535*(model_arr - model_arr.min())/(model_arr.max() - model_arr.min())))

In [ ]:
from tifffile import imwrite
sm_arr = np.array(sm)
imwrite("output/CNS_10obj_1e-4_random/CNS_10obj_1e-4_random_SVD_3_of_10comp.tif", np.float32(sm_arr))
imwrite("output/CNS_10obj_1e-4_random/CNS_10obj_1e-4_random_SVD_3_of_10comp_16bit.tif", np.uint16(65535*(sm_arr - sm_arr.min())/(sm_arr.max() - sm_arr.min())))

# PiecewiseAffineTransform

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import warp, PiecewiseAffineTransform
from skimage.feature import blob_log

canvas = img[0]# (img[0] - img[0].min()) / (img[0].max() - img[0].min())

# Generate random filled circles on a black canvas
canvas_size = canvas.shape
np.random.seed(0)

# Plot original image with random circles
plt.figure(figsize=(8, 6))
plt.imshow(canvas, cmap='gray')
plt.title('Original Image')
plt.axis('off')
plt.show()

# Identify control points by detecting corners
control_points = blob_log(canvas, max_sigma=5, num_sigma=10, threshold=.04, exclude_border = 50)[:,:2]

# Plot original image with control points
plt.figure(figsize=(8, 6))
plt.imshow(canvas, cmap='gray')
plt.plot(control_points[:, 1], control_points[:, 0], 'ro')
plt.title('Original Image with Control Points')
plt.axis('off')
plt.show()

# Assign each control point a random warping vector
warp_vectors = np.random.normal(0, 1, size=(len(control_points), 2))

# Create a PiecewiseAffineTransform object
transform = PiecewiseAffineTransform()

# Add control points and their corresponding displaced points to the transform object
src, dst = control_points, control_points + warp_vectors
transform.estimate(src, dst)

# Apply the warp transformation to the image
warped_image = warp(canvas, transform, output_shape=canvas.shape)

# Plot warped image
plt.figure(figsize=(8, 6))
plt.imshow(warped_image, cmap='gray')
plt.title('Warped Image')
plt.axis('off')
plt.show()

# Plot difference image
plt.figure(figsize=(8, 6))
plt.imshow(warped_image - canvas, cmap='seismic', vmin=-0.5, vmax=0.5)
plt.title('Difference Image')
plt.axis('off')
plt.show()
